In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Conv2D, DepthwiseConv2D, AveragePooling2D, Activation, SeparableConv2D, SpatialDropout1D, MaxPooling2D, Dot, Input, TimeDistributed, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from utils import preprocess_data
from mne.preprocessing import ICA

2023-09-15 00:53:16.208551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

'''
=========  ===================================
run        task
=========  ===================================
1          Baseline, eyes open
2          Baseline, eyes closed
3, 7, 11   Motor execution: left vs right hand
4, 8, 12   Motor imagery: left vs right hand
5, 9, 13   Motor execution: hands vs feet
6, 10, 14  Motor imagery: hands vs feet
=========  ===================================
'''
raws_train = []
raws_test = []
for ii in range(1, 10):
    subject = f'S{ii:03d}'
    files = glob.glob(f'../../files/{subject}/*.edf')
    for i in [5, 9, 13, 3, 7, 11]:
        current_file = files[i]
        r = read_raw_edf(current_file, preload=True, stim_channel='auto')
        events, _ = mne.events_from_annotations(r)
        if i in [5, 9, 13]:
            new_labels_events = {1:'rest', 2:'T1', 3:'T2'} # action
        elif i in [3, 7, 11]:
            new_labels_events = {1:'rest', 2:'T3', 3:'T4'}
        new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
        r.set_annotations(new_annot)
        if ii < 4:
            raws_test.append(r)
        else:
            raws_train.append(r)
    
raws_train_obj = concatenate_raws(raws_train)
raw_test_obj = concatenate_raws(raws_test)

Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/fil

In [3]:
def preprocess_raw(raw):
    # "Fc5.","Fc6.","Fc3.","Fc4.","Fc1.","Fc2.","C5..","C6..","C3..","C4..","C1..","C2..","Cp5.","Cp6.","Cp3.","Cp4.","Cp1.","Cp2."
    # raw = raw.pick_channels(['Fc5.', 'Fc6.', 'Fc3.', 'Fc4.', 'Fc1.', 'Fc2.', 'C5..', 'C6..', 'C3..', 'C4..', 'C1..', 'C2..', 'Cp5.', 'Cp6.', 'Cp3.', 'Cp4.', 'Cp1.', 'Cp2.'])

    # filters
    notch_freq = 60
    raw.notch_filter(notch_freq, fir_design='firwin')

    low_cutoff = 8
    high_cutoff = 40
    raw.filter(low_cutoff, high_cutoff, fir_design='firwin')

    events, event_dict = mne.events_from_annotations(raw)
    print(raw.info)
    print(event_dict)
    picks = mne.pick_types(raw.info, meg=True, eeg=True, stim=False, eog=False, exclude='bads')

    event_id = {'T1': 2, 'T2': 3, 'T3': 4, 'T4': 5}
    events, event_dict = mne.events_from_annotations(raw, event_id=event_id)
    tmin = -0.2  # Time before event in seconds
    tmax = 0.8  # Time after event in seconds
    epochs = mne.Epochs(raw, events, event_dict, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

    # raw, events, event_dict, picks, epochs
    return raw, events, event_dict, picks, epochs

In [4]:

raw_train = raws_train_obj.copy()
raw_test = raw_test_obj.copy()

raw_train, events_train, event_dict_train, picks_train, epochs_train = preprocess_raw(raw_train)
raw_test, events_test, event_dict_test, picks_test, epochs_test = preprocess_raw(raw_test)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1057 samples (6.606 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 36 contiguous segments


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    1.7s finished


Setting up band-pass filter from 8 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 265 samples (1.656 sec)



...

Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 8.0 Hz
 lowpass: 40.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'T1': 1, 'T2': 2, 'T3': 3, 'T4': 4, 'rest': 5}
Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4']
Not setting metadata
480 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 480 events and 161 original time points ...
0 bad epochs dropped
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequen

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 18 contiguous segments
Setting up band-pass filter from 8 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 265 samples (1.656 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 8.0 Hz
 lowpass: 40.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'T1': 1, 'T2': 2, 'T3': 3, 'T4': 4, 'rest': 5}
Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4']
Not setting metadata
255 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 255 events and 161 original time points ...
0 bad epochs dropped


In [5]:
X_train = epochs_train.get_data()
y_train = epochs_train.events[:, -1] - 2
y_train = to_categorical(y_train)

In [6]:
X_test = epochs_test.get_data()
y_test = epochs_test.events[:, -1] - 2
y_test = to_categorical(y_test)

In [7]:
# cnn
n_channels = X_train.shape[1]
input_window_size = X_train.shape[2]
input_shape = (1, n_channels, input_window_size)
X_train = X_train.reshape(X_train.shape[0], 1, n_channels, input_window_size)
X_test = X_test.reshape(X_test.shape[0], 1, n_channels, input_window_size)
print(X_train.shape, X_test.shape)

(480, 1, 64, 161) (255, 1, 64, 161)


In [16]:
model = Sequential()
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))

# Block 2: Spacial Convolution
model.add(Conv2D(16, (n_channels, 1), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(Dropout(0.25))

# Block 3: Separable Convolution
model.add(Conv2D(16, (1, 16), strides=(1, 1), padding='same', use_bias=False))
model.add(Conv2D(8, (1, 1), strides=(1, 1), padding='valid', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=(1, 8), strides=(1, 8)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

model.fit(X_train, y_train, epochs=250, batch_size=10, validation_data=(X_test, y_test), verbose=1, shuffle=False)
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

Epoch 1/250
48/48 [==============================] - 5s 55ms/step - loss: 1.5273 - accuracy: 0.2042 - val_loss: 1.4050 - val_accuracy: 0.2510
Epoch 2/250
48/48 [==============================] - 2s 38ms/step - loss: 1.4189 - accuracy: 0.2271 - val_loss: 1.3851 - val_accuracy: 0.2627
Epoch 3/250
48/48 [==============================] - 2s 39ms/step - loss: 1.4108 - accuracy: 0.2625 - val_loss: 1.3984 - val_accuracy: 0.2510
Epoch 4/250
48/48 [==============================] - 2s 36ms/step - loss: 1.4131 - accuracy: 0.2583 - val_loss: 1.3998 - val_accuracy: 0.2510
Epoch 5/250
48/48 [==============================] - 2s 36ms/step - loss: 1.3763 - accuracy: 0.3146 - val_loss: 1.4163 - val_accuracy: 0.2510
Epoch 6/250
48/48 [==============================] - 2s 37ms/step - loss: 1.3524 - accuracy: 0.3396 - val_loss: 1.4141 - val_accuracy: 0.2706
Epoch 7/250
48/48 [==============================] - 2s 36ms/step - loss: 1.3246 - accuracy: 0.3562 - val_loss: 1.4542 - val_accuracy: 0.2392
Epoch 

In [17]:
# cnn2d_classic

model = Sequential()
# Block 1: Temporal Convolution
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block 2: Spacial Convolution
model.add(Conv2D(int(n_channels/2), (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('elu'))
# model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(Dropout(0.25))

# Block 3: Separable Convolution
model.add(Conv2D(8, (1, 1), strides=(1, 1), padding='same', use_bias=False))
model.add(Conv2D(16, (1, 1), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('elu'))
# model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(Dropout(0.25))
model.add(Flatten())

# Classifier
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, epochs=250, batch_size=10, validation_data=(X_test, y_test), verbose=1, shuffle=False)
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_41 (Conv2D)          (None, 1, 64, 8)          82432     
                                                                 
 batch_normalization_29 (Bat  (None, 1, 64, 8)         32        
 chNormalization)                                                
                                                                 
 dropout_14 (Dropout)        (None, 1, 64, 8)          0         
                                                                 
 conv2d_42 (Conv2D)          (None, 1, 64, 32)         16384     
                                                                 
 batch_normalization_30 (Bat  (None, 1, 64, 32)        128       
 chNormalization)                                                
                                                                 
 activation_23 (Activation)  (None, 1, 64, 32)        

In [15]:
# gcn_classic

model = Sequential()
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))

# Block 2: Spacial Convolution
model.add(Conv2D(16, (n_channels, 1), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(Dropout(0.25))

# Block 3: Separable Convolution
model.add(Conv2D(16, (1, 16), strides=(1, 1), padding='same', use_bias=False))
model.add(Conv2D(8, (1, 1), strides=(1, 1), padding='valid', use_bias=False))
model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=(1, 8), strides=(1, 8)))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, epochs=250, batch_size=10, validation_data=(X_test, y_test), verbose=1, shuffle=False)
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 1, 64, 8)          82432     
                                                                 
 batch_normalization_17 (Bat  (None, 1, 64, 8)         32        
 chNormalization)                                                
                                                                 
 conv2d_28 (Conv2D)          (None, 1, 64, 8)          4096      
                                                                 
 batch_normalization_18 (Bat  (None, 1, 64, 8)         32        
 chNormalization)                                                
                                                                 
 activation_13 (Activation)  (None, 1, 64, 8)          0         
                                                                 
 conv2d_29 (Conv2D)          (None, 1, 64, 8)         

In [18]:
model = Sequential()
# Convolutional Layer 1
model.add(Conv2D(25, (1, n_channels), strides=(1, 1), input_shape=input_shape, padding='same', activation='elu'))

# Convolutional Layer 2
model.add(Conv2D(25, (1, 2), strides=(1, 1), padding='same', activation='elu'))

# Max Pooling Layer 1
model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

# Convolutional Layer 3
model.add(Conv2D(50, (11, 1), strides=(1, 1), padding='same', activation='elu'))

# Max Pooling Layer 2
model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

# Convolutional Layer 4
model.add(Conv2D(100, (11, 1), strides=(1, 1), padding='same', activation='elu'))

# Max Pooling Layer 3
model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding='same'))

# Convolutional Layer 5
model.add(Conv2D(200, (11, 1), strides=(1, 1), padding='same', activation='elu'))

# Max Pooling Layer 4
model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())

# Classifier
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, epochs=250, batch_size=10, validation_data=(X_test, y_test), verbose=1, shuffle=False)
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 1, 64, 25)         257625    
                                                                 
 conv2d_46 (Conv2D)          (None, 1, 64, 25)         1275      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 22, 25)        0         
 2D)                                                             
                                                                 
 conv2d_47 (Conv2D)          (None, 1, 22, 50)         13800     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 1, 8, 50)         0         
 2D)                                                             
                                                                 
 conv2d_48 (Conv2D)          (None, 1, 8, 100)        